In [ ]:
#directory
current_dir = !pwd
print(current_dir)

file_list = !ls
print(file_list)

#All files in ds
print(dataset_list)


#DF
import pandas as pd

# Read in `road-accidents.csv`
car_acc = pd.read_csv("datasets/road-accidents.csv", comment = "#", sep = "|")

# create tuple
rows_and_cols = car_acc.shape
print('There are {} rows and {} columns.\n'.format(
    rows_and_cols[0], rows_and_cols[1]))
#DataFrame
car_acc_information = car_acc.info()
print(car_acc_information)
#overview of data
import seaborn as sns
%matplotlib inline

sum_stat_car = car_acc.describe()
print(sum_stat_car)

#pairwise scatter plot to explore the data

sns.pairplot(car_acc)
#association of features and accidents
#correlation coefficent for all columns
corr_columns = car_acc.corr()
corr_columns
#linear regression
from sklearn import linear_model

#features and target DataFrames
features = car_acc[['perc_fatl_speed', 'perc_fatl_alcohol', 'perc_fatl_1st_time']]
target = car_acc['drvr_fatl_col_bmiles']

# linear regression object
reg = linear_model.LinearRegression()

# Fit model
reg.fit(features,target)
# regression coefficients
fit_coef = reg.coef_
fit_coef


# Standardize data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Import the PCA class and fit data
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(features_scaled)

# bar plot the proportion of variance
import matplotlib.pyplot as plt
plt.bar(range(1, pca.n_components_ + 1),  pca.explained_variance_ratio_)
plt.xlabel('Principal #')
plt.ylabel('Proportion of variance')
plt.xticks([1, 2, 3])

# cumulative proportion of variance from first two principal components
two_first_comp_var_exp = pca.explained_variance_ratio_.cumsum()[1]
print("The cumulative variance of the first two principal components is {}".format(
    round(two_first_comp_var_exp, 5)))
#visualize the principal components

# transform the scaled features using two principal components
pca = PCA(n_components=2)
p_comps = pca.fit_transform(features_scaled)

# extract the first and second component to use for the scatter plot
p_comp1 = p_comps[:, 0]
p_comp2 = p_comps[:, 1]

# scatter plot for the first two principal components
plt.scatter(p_comp1, p_comp2)
# finding clusters of data using KMeans

from sklearn.cluster import KMeans

ks = range(1, 10)
inertias = []
for k in ks:
    km = KMeans(n_clusters=k, random_state=8)
    km.fit(features_scaled)
    inertias.append(km.inertia_)
    
# line plot for results
plt.plot(ks, inertias, marker='o')

# create new KMeans object

km = KMeans(n_clusters = 3, random_state = 8)

# fit data

km.fit(features_scaled)

# scatter plot for the  2 components

plt.scatter(p_comps[:, 0], p_comps[:, 1], c=km.labels_)

#create visual that shows the difference between each cluster
# new column with the labels from the KMeans

car_acc['cluster'] = km.labels_

# creating a long format df to showcase a violin plot

melt_car = pd.melt(car_acc, id_vars='cluster', var_name='measurement', value_name='percent',
                   value_vars=['perc_fatl_speed', 'perc_fatl_alcohol', 'perc_fatl_1st_time'])
sns.violinplot(y='measurement', x='percent', data=melt_car, hue='cluster')

# read in new dataset and merge with previous df
miles_driven = pd.read_csv('datasets/miles-driven.csv', sep='|')
car_acc_miles = car_acc.merge(miles_driven, on='state')

# make a new column for the number of drivers involved in fatal accidents amd show results in a barplot
car_acc_miles['num_drvr_fatl_col'] = car_acc_miles['drvr_fatl_col_bmiles'] * car_acc_miles['million_miles_annually'] / 1000

sns.barplot(x='cluster', y='num_drvr_fatl_col', data=car_acc_miles, estimator=sum, ci=None)

# calculate count,mean,sum of the number of states in each cluster for the col 'num_drvr_fatl_col'
count_mean_sum = car_acc_miles.groupby('cluster')['num_drvr_fatl_col'].agg(['count', 'mean', 'sum'])
count_mean_sum
#understanding the data derived from the barplot as col 0 being the most significant cluster 